In [44]:
import networkx
import torch
import numpy as np
import pandas as pd
from sklearn.metrics import *
from torch_geometric.loader import NeighborSampler, NeighborLoader
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GATConv, ResGatedGraphConv, GATv2Conv, SAGEConv, GENConv, DeepGCNLayer, PairNorm, GINConv
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import torch.nn.functional as F
from imblearn.under_sampling import RandomUnderSampler
pd.options.mode.use_inf_as_na = True
from collections import Counter
from sklearn.feature_selection import SelectFromModel
import torch.nn as nn
import time
import pickle
from torch.nn import LayerNorm, Linear, ReLU
from torch_scatter import scatter
from tqdm import tqdm
from torch_geometric.loader import RandomNodeSampler
import math
import copy
from sklearn.metrics import f1_score
from torch.optim import lr_scheduler

In [45]:
import warnings
# action参数可以设置为ignore，一位一次也不喜爱你是，once表示为只显示一次
warnings.filterwarnings(action='ignore')

In [46]:
np.random.seed(2022)
torch.manual_seed(2022)

In [47]:
class Transition_layer(torch.nn.Module):
    def __init__(self, act, norm, lin):
        super().__init__()
        self.act = act
        self.norm = norm
        self.lin = lin
    def forward(self, x):
        x = self.norm(x)
        x = self.act(x)
        x = self.lin(x)
        return x
        
class DenseGAT(torch.nn.Module):
    def __init__(self, in_channels, num_class, num_layers=6, num_blocks=3, growth_rate=10, theta=0.5):
        super().__init__()
        self.num_layers = num_layers
        self.blocks = num_blocks
        self.theta = theta
        self.growth_rate = growth_rate
        self.base_rate = growth_rate
        self.in_channels = in_channels
        self.out_channels = num_class
        self.linear_layers = torch.nn.ModuleList()
        self.transition_layers = torch.nn.ModuleList()
        self.block_layers = torch.nn.ModuleList()
        self.node_encoder = Linear(in_channels, in_channels)
        self.linear_layers.append(self.node_encoder)
        
        for i in range(self.blocks):
            # block
            layers = torch.nn.ModuleList()
            # 2^(i - 1) * k0
            self.growth_rate = int(math.pow(2, i) * self.base_rate)
            print(self.growth_rate)
            for j in range(1, self.num_layers + 1):
                conv = GATv2Conv(in_channels + (j - 1) * self.growth_rate, self.growth_rate, aggr='max') 
                norm = LayerNorm(self.growth_rate)
                act = ReLU()
                layer = DeepGCNLayer(conv, norm, act, block='dense')
                layers.append(layer)
            self.block_layers.append(layers)
            
            # transition
            hidden_channels = in_channels +  self.num_layers * self.growth_rate
            out_channels = int(hidden_channels * self.theta)
            transition_norm = LayerNorm(hidden_channels, elementwise_affine=True)
            transition_act = ReLU()
            transition_lin = Linear(hidden_channels, out_channels)
            transitionLayer = Transition_layer(transition_act, transition_norm, transition_lin)
            self.transition_layers.append(transitionLayer)
            in_channels = copy.copy(out_channels)
        
        self.lin_last = Linear(in_channels, self.out_channels)
        self.linear_layers.append(self.lin_last)
        
    def forward(self, x, edge_index):
#         x = self.linear_layers[0](x)
        for i in range(self.blocks):
            # block layer
            for layer in self.block_layers[i]:
                x = layer(x, edge_index)
            # transition layer
            x = self.transition_layers[i](x)
        x = self.linear_layers[-1](x)
        return x

In [48]:
def train():
    total_loss = total_correct = total_examples = 0
    start_time = time.time()
    for batch in train_loader:
        batch = batch.to(device)
        y = batch.y[:batch.batch_size]
        optimizer.zero_grad()
        y_hat = model(batch.x.to(device), batch.edge_index.to(device))[:batch.batch_size]
        loss = F.cross_entropy(y_hat, y)
        loss.backward()
        optimizer.step()
#         scheduler1.step()
        total_loss += float(loss) * batch.batch_size
        total_correct += int((y_hat.argmax(dim=-1) == y).sum())
        total_examples += batch.batch_size
    end_time = time.time()  
    
    return total_loss / total_examples, total_correct / total_examples

def inferrence(model, subgraph_loader):
    total_loss = total_correct = total_examples = 0
    xs = []
    y = []
    pbar = tqdm(total=len(subgraph_loader))
    for batch in subgraph_loader:
        y_hat = model(batch.x, batch.edge_index.to(device))[:batch.batch_size]
        xs.append(y_hat.cpu())
        y_true = batch.y[:batch.batch_size]
        y.append(y_true.cpu())
        pbar.update(1)
    pbar.close()
    y_hat = torch.cat(xs, 0)
    y_hat = y_hat.detach().numpy()
    y_hat = np.argmax(y_hat, -1)
    y = torch.cat(y, 0)
    print(y_hat)
    print(y)
    cr1 = classification_report(y, y_hat,digits=4)
    cf = confusion_matrix(y, y_hat)
    print(cr1)
    print(cf)


In [49]:
train_data = torch.load('cic2017_directed_line_graph_train_data_15label')

In [50]:
test_data = torch.load('cic2017_directed_line_graph_test_data_15label')

In [8]:
all_data = torch.load(r'/home/xiaoyujie/densegat/cic2017_directed_lg_data_15label')

In [51]:
train_data,test_data, all_data

(Data(x=[161162, 78], edge_index=[2, 78210208], y=[161162], num_nodes=161162),
 Data(x=[28441, 78], edge_index=[2, 2293931], y=[28441], num_nodes=28441),
 Data(x=[189603, 78], edge_index=[2, 108609698], y=[189603], num_nodes=189603, train_mask=[161162], test_mask=[28441]))

In [10]:
# 二分类试试

In [11]:
# train_label = train_data.y.numpy()
# test_label = test_data.y.numpy()
# train_label[train_label != 0] = 1
# test_label[test_label != 0] = 1
# train_data.y = torch.LongTensor(train_label)
# test_data.y = torch.LongTensor(test_label)

In [52]:
train_num_nodes = torch.arange(len(train_data.y))
test_num_nodes = torch.arange(len(test_data.y))

In [53]:
train_data = train_data.to('cpu')
test_data = test_data.to('cpu')

In [54]:
node_num_per_k = 50
depth = 5

In [55]:
hop = [node_num_per_k] * depth
train_loader = NeighborLoader(train_data, input_nodes = train_num_nodes, num_neighbors=hop, batch_size=1024, shuffle=True)

In [56]:
next(iter(train_loader))

Data(x=[16180, 78], edge_index=[2, 46587], y=[16180], num_nodes=16180, batch_size=1024)

In [57]:
subgraph_loader = NeighborLoader(test_data, input_nodes=test_num_nodes,num_neighbors=hop, batch_size=1024, shuffle=False)

In [58]:
in_channels = train_data.x.size(-1)
num_class = 15
model = DenseGAT(in_channels=in_channels, num_class=num_class, 
                 num_layers=10, num_blocks=1, growth_rate=10, theta=0.8)
loss_all = []

10


In [59]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [60]:
device = 'cuda:3'
model = model.to(device)

In [61]:
model.train()
epochs = 500
for i in range(1, 1 + epochs):
#     train_loader = NeighborLoader(train_data, input_nodes = train_num_nodes, num_neighbors=hop, batch_size=1024, shuffle=True)
    start_time = time.time()
    loss, acc = train()
    end_time = time.time()
    if i % 50:
        torch.save(model.state_dict(), 'cic_epoch' + str(i))
    print('epoch: {:04d}'.format(i),
          'loss_train: {:.4f}'.format(loss),
          'acc_train: {:.4f}'.format(acc),
          'time: {:.4f}s'.format(end_time - start_time))
    loss_all.append(loss)

epoch: 0001 loss_train: 0.2071 acc_train: 0.9322 time: 9.9575s
epoch: 0002 loss_train: 0.1048 acc_train: 0.9641 time: 11.1570s
epoch: 0003 loss_train: 0.0997 acc_train: 0.9660 time: 9.8670s
epoch: 0004 loss_train: 0.0909 acc_train: 0.9685 time: 9.8009s
epoch: 0005 loss_train: 0.0760 acc_train: 0.9725 time: 9.6376s
epoch: 0006 loss_train: 0.0948 acc_train: 0.9677 time: 9.5538s
epoch: 0007 loss_train: 0.0793 acc_train: 0.9714 time: 9.6720s
epoch: 0008 loss_train: 0.0681 acc_train: 0.9746 time: 9.1248s
epoch: 0009 loss_train: 0.0711 acc_train: 0.9740 time: 9.1989s
epoch: 0010 loss_train: 0.0692 acc_train: 0.9746 time: 9.1313s
epoch: 0011 loss_train: 0.0644 acc_train: 0.9762 time: 10.0006s


KeyboardInterrupt: 

In [62]:
device = 'cpu'
model = model.to(device)

In [63]:
y_hat = model(test_data.x.to(device), test_data.edge_index.to(device))
y_hat = y_hat.detach().numpy()
y_hat = np.argmax(y_hat, -1)

In [64]:
test_y = test_data.y.numpy()

In [65]:
cr = classification_report(test_y, y_hat, output_dict=True)
df = pd.DataFrame(cr).transpose()
df

,precision,recall,f1-score,support
0,0.993342,0.950370,0.971381,13500.000000
1,0.993369,0.998667,0.996011,3000.000000
2,0.995346,0.998000,0.996671,3000.000000
3,0.686717,0.935154,0.791908,293.000000
4,0.750000,0.600000,0.666667,5.000000
5,0.983471,1.000000,0.991667,1190.000000
6,0.849462,0.981921,0.910901,885.000000
7,0.984813,0.968966,0.976825,870.000000
8,0.972586,0.989091,0.980769,825.000000
9,0.908123,0.995000,0.949578,3000.000000


In [176]:
cr = classification_report(test_y, y_hat, output_dict=True)
df = pd.DataFrame(cr).transpose()
df

,precision,recall,f1-score,support
0,0.899637,0.918296,0.908871,13500.000000
1,0.924766,0.907436,0.916019,14941.000000
accuracy,0.912591,0.912591,0.912591,0.912591
macro avg,0.912202,0.912866,0.912445,28441.000000
weighted avg,0.912838,0.912591,0.912626,28441.000000


In [30]:
cr = classification_report(test_y, y_hat, output_dict=True)
df = pd.DataFrame(cr).transpose()
df

,precision,recall,f1-score,support
0,0.523932,0.952333,0.675973,3000.000000
1,0.999584,0.801000,0.889341,3000.000000
2,0.953615,0.993667,0.973229,3000.000000
3,0.972222,0.358362,0.523691,293.000000
4,0.000000,0.000000,0.000000,5.000000
5,0.998724,0.657983,0.793313,1190.000000
6,0.000000,0.000000,0.000000,885.000000
7,0.974114,0.821839,0.891521,870.000000
8,0.985384,0.980606,0.982989,825.000000
9,0.989789,0.937000,0.962671,3000.000000


In [30]:
inferrence(model, subgraph_loader)

100%|██████████| 28/28 [00:00<00:00, 34.64it/s]


[2 0 0 ... 0 0 0]
tensor([2, 0, 0,  ..., 0, 0, 0])
              precision    recall  f1-score   support

           0     0.8596    0.9609    0.9075     13500
           1     0.9931    0.9647    0.9787      3000
           2     0.9226    0.9927    0.9563      3000
           3     0.6329    0.9590    0.7626       293
           4     1.0000    0.2000    0.3333         5
           5     0.9838    0.5109    0.6726      1190
           6     0.6667    0.0045    0.0090       885
           7     0.9647    0.9747    0.9697       870
           8     0.9794    0.9818    0.9806       825
           9     0.9849    0.8673    0.9224      3000
          10     0.9980    0.9462    0.9714      1544
          11     0.0000    0.0000    0.0000         2
          12     0.5710    0.8186    0.6727       226
          13     0.0000    0.0000    0.0000        98
          14     0.0000    0.0000    0.0000         3

    accuracy                         0.9017     28441
   macro avg     0.7038    0.

In [107]:
inferrence(model, subgraph_loader)


100%|██████████| 22/22 [00:00<00:00, 32.48it/s]


[0 1 2 ... 1 0 2]
tensor([0, 1, 2,  ..., 1, 0, 2])
              precision    recall  f1-score   support

           0     0.9778    0.9412    0.9592      7500
           1     0.9793    0.9893    0.9843      9240
           2     0.9652    0.9890    0.9770      3000
           3     0.8594    0.9354    0.8958       294
           4     1.0000    0.4000    0.5714         5
           5     0.9448    0.9889    0.9663      2075
           6     0.9156    0.8960    0.9057       327

    accuracy                         0.9709     22441
   macro avg     0.9489    0.8771    0.8942     22441
weighted avg     0.9712    0.9709    0.9709     22441

[[7059  173  104   45    0   94   25]
 [  95 9141    0    0    0    3    1]
 [  25    7 2967    0    0    0    1]
 [  19    0    0  275    0    0    0]
 [   2    1    0    0    2    0    0]
 [  15    6    2    0    0 2052    0]
 [   4    6    1    0    0   23  293]]


In [158]:
inferrence(model, subgraph_loader)

100%|██████████| 28/28 [00:01<00:00, 17.11it/s]


[1 0 0 ... 0 0 0]
tensor([1, 0, 0,  ..., 0, 0, 0])
              precision    recall  f1-score   support

           0     0.8286    0.9610    0.8899     13500
           1     0.9589    0.8204    0.8842     14941

    accuracy                         0.8871     28441
   macro avg     0.8937    0.8907    0.8871     28441
weighted avg     0.8970    0.8871    0.8869     28441

[[12974   526]
 [ 2684 12257]]


In [523]:
inferrence(model, subgraph_loader)

100%|██████████| 22/22 [00:05<00:00,  4.05it/s]

[0 1 2 ... 1 0 2]
tensor([0, 1, 2,  ..., 1, 0, 2])
              precision    recall  f1-score   support

           0     0.9142    0.9667    0.9397      7500
           1     0.9942    0.9661    0.9800      9240
           2     0.9657    0.9753    0.9705      3000
           3     0.8132    0.9626    0.8816       294
           4     1.0000    0.6000    0.7500         5
           5     0.9780    0.8578    0.9140      2075
           6     0.9366    0.9480    0.9422       327

    accuracy                         0.9571     22441
   macro avg     0.9431    0.8967    0.9111     22441
weighted avg     0.9590    0.9571    0.9573     22441

[[7250   45  104   65    0   17   19]
 [ 312 8927    0    0    0    0    1]
 [  51    1 2926    0    0   21    1]
 [  11    0    0  283    0    0    0]
 [   1    1    0    0    3    0    0]
 [ 292    3    0    0    0 1780    0]
 [  13    2    0    0    0    2  310]]


In [515]:
inferrence(model, subgraph_loader)

100%|██████████| 22/22 [00:00<00:00, 24.86it/s]


[0 1 2 ... 1 0 2]
tensor([0, 1, 2,  ..., 1, 0, 2])
              precision    recall  f1-score   support

           0     0.9908    0.9437    0.9667      7500
           1     0.9826    0.9963    0.9894      9240
           2     0.9646    0.9890    0.9766      3000
           3     0.8304    0.9660    0.8931       294
           4     1.0000    0.6000    0.7500         5
           5     0.9462    0.9908    0.9680      2075
           6     0.9162    0.9358    0.9259       327

    accuracy                         0.9759     22441
   macro avg     0.9472    0.9174    0.9242     22441
weighted avg     0.9766    0.9759    0.9759     22441

[[7078  148  105   58    0   86   25]
 [  18 9206    0    0    0   15    1]
 [  20    7 2967    0    0    5    1]
 [  10    0    0  284    0    0    0]
 [   1    1    0    0    3    0    0]
 [  11    4    3    0    0 2056    1]
 [   6    3    1    0    0   11  306]]


In [463]:
inferrence(model, subgraph_loader)


100%|██████████| 18/18 [00:00<00:00, 30.26it/s]

[2 1 6 ... 1 1 7]
tensor([2, 1, 6,  ..., 1, 1, 7])
              precision    recall  f1-score   support

           0     0.7709    0.9249    0.8409      2998
           1     0.9997    0.9750    0.9872      3000
           2     0.9800    0.9987    0.9893      2998
           3     0.7153    0.7201    0.7177       293
           4     0.0769    0.4000    0.1290         5
           5     0.7298    0.1748    0.2820      1190
           6     0.3454    0.5186    0.4146       885
           7     0.9564    0.9069    0.9310       870
           8     0.9768    0.9697    0.9732       825
           9     0.9665    0.9454    0.9558      2987
          10     0.9980    0.9812    0.9895      1544
          11     0.0000    0.0000    0.0000         2

    accuracy                         0.8808     17597
   macro avg     0.7096    0.7096    0.6842     17597
weighted avg     0.8921    0.8808    0.8727     17597

[[2773    1   60   83   24    7   13    5    6   24    2    0]
 [   2 2925    0   

In [538]:
y_hat = model(test_data.x.to(device), test_data.edge_index.to(device))

In [539]:
y_hat = y_hat.detach().numpy()
y_hat = np.argmax(y_hat, -1)

In [540]:
len(y_hat)

22441

In [541]:
classification_report(test_y, y_hat, digits=4)

'              precision    recall  f1-score   support\n\n           0     0.9189    0.9665    0.9421      7500\n           1     0.9941    0.9661    0.9799      9240\n           2     0.9649    0.9907    0.9776      3000\n           3     0.8109    0.9626    0.8802       294\n           4     1.0000    0.6000    0.7500         5\n           5     0.9873    0.8607    0.9197      2075\n           6     0.9366    0.9480    0.9422       327\n\n    accuracy                         0.9594     22441\n   macro avg     0.9447    0.8992    0.9131     22441\nweighted avg     0.9612    0.9594    0.9595     22441\n'

In [137]:
y_hat = model(all_data.x.to(device), all_data.edge_index.to(device))

  0%|          | 0/186 [46:03<?, ?it/s]


RuntimeError: [enforce fail at CPUAllocator.cpp:68] . DefaultCPUAllocator: can't allocate memory: you tried to allocate 8703520240 bytes. Error code 12 (Cannot allocate memory)

In [ ]:
y_hat = y_hat.detach().numpy()
y_hat = np.argmax(y_hat, -1)

In [120]:
classification_report(test_y, y_hat[all_data.test_mask], digits=4)

IndexError: index 171153 is out of bounds for axis 0 with size 28441

In [96]:
from tqdm import tqdm

In [138]:
subgraph_loader2 = NeighborLoader(copy.copy(all_data), input_nodes=None,num_neighbors=[-1], batch_size=1024, shuffle=False)
subgraph_loader2.data.num_nodes = all_data.num_nodes
subgraph_loader2.data.n_id = torch.arange(all_data.num_nodes)

In [139]:
def inferrence2(model, subgraph_loader):
    pbar = tqdm(total=len(subgraph_loader))
    for i, batch in enumerate(subgraph_loader):
        batch = batch.to(device)
        y_hat = model(batch.x.to(device), batch.edge_index.to(device))[:batch.batch_size]
        np.save('unsw_yhat/y_hat_batch' + str(i), y_hat.detach().numpy())
        pbar.update(1)
    pbar.close()

In [140]:
def test(data, loader):
    xs = []
    for i in range(len(loader)):
        y_hat = np.load('unsw_yhat/y_hat_batch' + str(i) + '.npy')
        xs.append(y_hat)
    y_hat = np.concatenate(xs, 0)
    y_hat = np.argmax(y_hat, -1)
    y = data.y.to(device)
    test_mask = data.test_mask 
    f1 = f1_score(y[test_mask], y_hat[test_mask], average='weighted')
    pr = precision_score(y[test_mask], y_hat[test_mask], average='weighted')
    cr1 = classification_report(y[test_mask], y_hat[test_mask], digits=4)
    cf = confusion_matrix(y[test_mask], y_hat[test_mask])
    print("测试集结果：")
    print(cr1)
    print(cf)
    
    train_mask = data.train_mask
    f1 = f1_score(y[train_mask], y_hat[train_mask], average='weighted')
    pr = precision_score(y[train_mask], y_hat[train_mask], average='weighted')
    cr1 = classification_report(y[train_mask], y_hat[train_mask], digits=4)
    cf = confusion_matrix(y[train_mask], y_hat[train_mask])
    print("训练集结果：")
    print(cr1)
    print(cf)
    return f1, pr

In [141]:
device = 'cpu'

In [142]:
model = model.to(device)

In [159]:
inferrence2(model, subgraph_loader2)

 55%|█████▍    | 102/186 [08:03<17:47, 12.71s/it]

KeyboardInterrupt: 

In [ ]:
test(all_data, subgraph_loader2)

In [135]:
all_data_label = all_data.y.numpy()

In [136]:
all_data_label[all_data_label != 0] = 1
all_data.y = torch.LongTensor(all_data_label)

In [109]:
all_data

Data(x=[189603, 78], edge_index=[2, 108609698], y=[189603], num_nodes=189603, train_mask=[161162], test_mask=[28441])

In [110]:
subgraph_loader2 = NeighborLoader(all_data, input_nodes=all_data.test_mask,
                                 num_neighbors=hop, batch_size=500, shuffle=False)

In [73]:
train_data = train_data.to('cpu')
test_data = test_data.to('cpu')

In [74]:
x_train = train_data.x.numpy()
y_train = train_data.y.numpy()

In [75]:
x_test = test_data.x.numpy()
y_test = test_data.y.numpy()

In [76]:
from catboost import CatBoostClassifier

In [77]:
from sklearn.ensemble._forest import RandomForestClassifier

In [90]:
cbc = CatBoostClassifier(n_estimators=100)

In [91]:
cbc.fit(x_train, y_train)

Learning rate set to 0.5
0:	learn: 0.4805458	total: 55ms	remaining: 5.45s
1:	learn: 0.3225481	total: 87.9ms	remaining: 4.31s
2:	learn: 0.2275914	total: 121ms	remaining: 3.91s
3:	learn: 0.1780775	total: 154ms	remaining: 3.69s
4:	learn: 0.1261871	total: 190ms	remaining: 3.61s
5:	learn: 0.1100746	total: 220ms	remaining: 3.45s
6:	learn: 0.0897102	total: 252ms	remaining: 3.34s
7:	learn: 0.0745446	total: 287ms	remaining: 3.3s
8:	learn: 0.0665954	total: 321ms	remaining: 3.24s
9:	learn: 0.0586550	total: 357ms	remaining: 3.21s
10:	learn: 0.0536122	total: 387ms	remaining: 3.13s
11:	learn: 0.0494401	total: 428ms	remaining: 3.14s
12:	learn: 0.0446124	total: 465ms	remaining: 3.11s
13:	learn: 0.0434772	total: 492ms	remaining: 3.02s
14:	learn: 0.0411160	total: 526ms	remaining: 2.98s
15:	learn: 0.0390528	total: 555ms	remaining: 2.91s
16:	learn: 0.0379762	total: 585ms	remaining: 2.86s
17:	learn: 0.0357098	total: 614ms	remaining: 2.79s
18:	learn: 0.0334713	total: 646ms	remaining: 2.75s
19:	learn: 0.0319

In [92]:
y_pred = cbc.predict(x_test)

In [93]:
classification_report(y_test, y_pred, digits=4)

'              precision    recall  f1-score   support\n\n           0     0.4211    0.9788    0.5889      7500\n           1     0.9154    0.4870    0.6358      9240\n           2     1.0000    0.0007    0.0013      3000\n           3     0.8409    0.2517    0.3874       294\n           4     1.0000    0.4000    0.5714         5\n           5     0.0000    0.0000    0.0000      2075\n           6     0.0000    0.0000    0.0000       327\n\n    accuracy                         0.5311     22441\n   macro avg     0.5968    0.3026    0.3121     22441\nweighted avg     0.6626    0.5311    0.4640     22441\n'